# LLM 미세 조정

## 데이터셋 가져오기

In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [2]:
dataset["train"][100]


{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

## 토큰화

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\yong\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yong\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
tokenized_datasets['train'][100]['input_ids']

[101,
 12008,
 27788,
 2523,
 119,
 151,
 9435,
 11455,
 119,
 133,
 9304,
 120,
 135,
 133,
 9304,
 120,
 135,
 1636,
 12058,
 1132,
 2066,
 17355,
 9860,
 119,
 1109,
 2523,
 1108,
 2213,
 119,
 2009,
 146,
 1138,
 1106,
 7380,
 1113,
 1115,
 146,
 1274,
 112,
 189,
 1221,
 119,
 1188,
 1110,
 1640,
 170,
 5671,
 1104,
 1139,
 1159,
 119,
 146,
 1198,
 1458,
 1106,
 11857,
 1639,
 119,
 138,
 6005,
 2386,
 1142,
 2523,
 119,
 1109,
 3176,
 22797,
 1105,
 1103,
 2269,
 1110,
 1198,
 182,
 14824,
 7770,
 119,
 6304,
 1107,
 1451,
 1236,
 119,
 1109,
 1178,
 3505,
 1645,
 1164,
 1103,
 2523,
 1132,
 14760,
 9368,
 138,
 19610,
 2315,
 112,
 188,
 13016,
 119,
 2431,
 1115,
 1108,
 9832,
 1463,
 1118,
 170,
 6434,
 1105,
 8362,
 23063,
 4902,
 9372,
 2741,
 119,
 1109,
 2523,
 1110,
 170,
 9874,
 14255,
 19091,
 5790,
 1105,
 5733,
 8362,
 26438,
 2727,
 1104,
 14946,
 119,
 133,
 9304,
 120,
 135,
 133,
 9304,
 120,
 135,
 10899,
 1208,
 146,
 1821,
 1198,
 1280,
 1106,
 26133,
 1113,
 

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

## BERT 모델 인스턴스화

In [6]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

### 평가 지표

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs = 2, eval_strategy="epoch")

### 학습 및 저장

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\yong\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.7369823455810547, 'eval_accuracy': 0.48, 'eval_runtime': 0.7678, 'eval_samples_per_second': 260.496, 'eval_steps_per_second': 32.562, 'epoch': 1.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.6642929315567017, 'eval_accuracy': 0.635, 'eval_runtime': 0.8513, 'eval_samples_per_second': 234.934, 'eval_steps_per_second': 29.367, 'epoch': 2.0}
{'train_runtime': 9.0676, 'train_samples_per_second': 44.113, 'train_steps_per_second': 5.514, 'train_loss': 0.6798026275634765, 'epoch': 2.0}


TrainOutput(global_step=50, training_loss=0.6798026275634765, metrics={'train_runtime': 9.0676, 'train_samples_per_second': 44.113, 'train_steps_per_second': 5.514, 'total_flos': 105244422144000.0, 'train_loss': 0.6798026275634765, 'epoch': 2.0})

In [13]:
trainer.save_model('models/sentiment-classifier')

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
trainer.push_to_hub("valen/sentiment-classifier")

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ychoikr/test_trainer/commit/a691082e19bf5284998d2369fa9d8a88aaaeddb8', commit_message='valen/sentiment-classifier', commit_description='', oid='a691082e19bf5284998d2369fa9d8a88aaaeddb8', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
from transformers import BertConfig, BertModel

model = AutoModelForSequenceClassification.from_pretrained('models/sentiment-classifier')

In [17]:
inputs = tokenizer("I cannot stand it anymore!", return_tensors="pt")

outputs = model(**inputs)

In [18]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.0131, 0.0990]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [19]:
import tensorflow as tf


predictions = tf.math.softmax(outputs.logits.detach(), axis=-1)
print(predictions)

tf.Tensor([[0.47853583 0.5214642 ]], shape=(1, 2), dtype=float32)


In [20]:
from transformers import pipeline

classifier = pipeline(task ='sentiment-analysis', model = model, tokenizer = tokenizer)
classifier('I cannot believe you did it again!')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.5391558408737183}]